<a href="https://colab.research.google.com/github/nandwanatarun/EVA-DeepLearning/blob/Session-19/S19_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
IN_COLAB = 'google.colab' in sys.modules
from getpass import getpass
import os

#user = getpass('BitBucket user')
#password = getpass('BitBucket password')
#os.environ['BITBUCKET_AUTH'] = user + ':' + password
#!git clone https://$BITBUCKET_AUTH@bitbucket.org/{user}/repository.git
!git clone https://github.com/nandwanatarun/test.git
!ls test/
#import https://github.com/nandwanatarun/EVA-DeepLearning/tree/Session-19/data/cars.git

#path  = "/Session-19" 
#os.chdir(path)


fatal: destination path 'test' already exists and is not an empty directory.
data  faceBlendCommon.py  generator_model_10000.h5  models  videos


In [8]:

import cv2
import os
import numpy as np
from os import listdir
from os.path import join

car_types = ['hatch', 'sedan', 'suv']

def resize_image(img, size=(64,64)):

    h, w = img.shape[:2]

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        c = img.shape[2]
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

spath, dpath = join('test/data', 'cars'), join('test/data', 'norm')

#if not os.path.isdir(dpath):
#    os.mkdir(dpath)

imgs = {}


for ct in car_types:
    sp, dp = join(spath, ct), join(dpath, ct)
    alen = 1024
    imgs[ct] = np.zeros((1024, 64, 64, 3))
 #   if not os.path.isdir(dp):
 #       os.mkdir(dp)
    tlen = 0
    for f in listdir(sp):
        sf, df = join(sp, f), join(dp, f)
        img = cv2.imread(sf)
        img = resize_image(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        tlen += 1
        if tlen > alen:
            imgs[ct] = np.append(imgs[ct], np.zeros((1024, 64, 64, 3)))
            alen += 1024
        imgs[ct][tlen-1] = img
    imgs[ct] = imgs[ct][:tlen]
    print(imgs[ct].shape)

(332, 64, 64, 3)
(410, 64, 64, 3)
(337, 64, 64, 3)


In [9]:
#%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainx, testx = imgs['hatch'][:300,:,:,:], imgs['hatch'][300:,:,:,:]

trainy, testy = np.zeros(trainx.shape[0], dtype=float), np.zeros(testx.shape[0], dtype=float)

trainx = np.append(trainx, imgs['sedan'][:375,:,:,:], axis=0)
testx = np.append(testx, imgs['sedan'][375:,:,:,:], axis=0)

ltrain, ltest = trainx.shape[0] - trainy.shape[0], testx.shape[0] - testy.shape[0]

trainy, testy = np.append(trainy, np.ones(ltrain, dtype=float)), np.append(testy, np.ones(ltest, dtype=float))


print(trainx.shape, "y", trainy.shape)
print(testx.shape, 'y', testy.shape)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(trainx)

trainX, trainY = trainx, trainy
testX, testY   = testx, testy

for i in range(9):
    iterate = datagen.flow(trainx, trainy, batch_size=len(trainx), shuffle=True)
    x, y = iterate.next()
    trainX, trainY = np.append(trainX, x, axis=0), np.append(trainY, y, axis=0)

for i in range(9):
    iterate = datagen.flow(testx, testy, batch_size=len(testx), shuffle=True)
    x, y = iterate.next()
    testX, testY = np.append(testX, x, axis=0), np.append(testY, y, axis=0)


print(trainX.shape, "y", trainY.shape)
print(testX.shape, "y", testY.shape)

trainx, trainy = trainX, trainY
testx, testy   = testX, testY

trainx = trainx.astype('float32') / 255
testx  = testx.astype('float32') / 255

trainx_mean = np.mean(trainx, axis=(0, 1, 2))
trainx_std  = np.std(trainx, axis=(0, 1, 2))

print(trainx_mean, trainx_std)

trainx -= trainx_mean
trainx /= trainx_std

testx -= trainx_mean
testx /= trainx_std

trainX, trainY = trainx, trainy #utils.to_binary(trainy)
testX,  testY  = testx,  testy  #utils.to_binary(testy)

min_pix, max_pix = trainX.min(), trainX.max()

print(min_pix, max_pix)
print(testX.min(), testX.max())

print(trainX.shape, trainY.shape)

1.15.0
(675, 64, 64, 3) y (675,)
(67, 64, 64, 3) y (67,)
(6750, 64, 64, 3) y (6750,)
(670, 64, 64, 3) y (670,)
[0.35469517 0.34651023 0.34183276] [0.38624603 0.3320597  0.3347992 ]
-1.0435178 1.9679887
-1.0435178 1.9679887
(6750, 64, 64, 3) (6750,)


In [10]:
print(testY)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1.

In [0]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import add, Input, Dense, Flatten, GlobalAvgPool2D, GlobalAvgPool1D
from tensorflow.keras.initializers import zeros

def ResConv(x, kernel=(3, 3), depth=32, maxpool=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if maxpool :
        x = MaxPool2D()(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def ResUnit(x, depth=32, maxpool=False):
    x = ResConv(x, depth=depth, maxpool=maxpool)
    x = ResConv(x, depth=depth)
    return x
    
def ResNetBlock(x, nunit, depth=32, maxpool=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    if maxpool:
        xskip = Conv2D(depth, (1, 1), strides=2, use_bias=False)(x)
    else: 
        xskip = x
    x = add([ResUnit(x, depth=depth, maxpool=maxpool), xskip])
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = add([ResUnit(x, depth=depth), x])
        x = add([ResUnit(x, depth=depth), x])
    return x

# Returns latent vector of 32 bytes
def ResNet9(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 3, 2)
    maxpools = (False, True, True)
    depths   = (64, 32, 32)
    
    for i in range(3):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i])
    x = GlobalAvgPool2D()(x)
    return x

#def D_init():
#    xin = Input(shape=(64, 64, 3))
#    x = ResNet9(xin)
#    x = Dense(1, use_bias=False, activation='sigmoid')(x)
#    return Model(xin, x)

def Q_init():
    xin = Input(shape=(64, 64, 3))
    x = ResNet9(xin)
    return Model(xin, x)


def D_init():
    xin = Input(shape=(64, 64, 3))
    x = Flatten()(xin)
    x = Dense(1, activation='sigmoid')(x)
    return Model(xin, x)

In [0]:
from tensorflow.keras import Model
from tensorflow.keras.layers import UpSampling2D, Reshape, Conv2DTranspose

def InvResConv(x, kernel=(3, 3), depth=32, upscale=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if upscale :
        x = UpSampling2D((2,2))(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def InvResUnit(x, depth=32, upscale=False):
    x = InvResConv(x, depth=depth)
    x = InvResConv(x, depth=depth, upscale=upscale)
    return x
    
def InvResNetBlock(x, nunit, depth=32, upscale=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    x = InvResUnit(x, depth=depth, upscale=upscale)
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = InvResUnit(x, depth=depth)
        x = InvResUnit(x, depth=depth)
    return x


def InvResNet9(x, size=(4,4)):
    depths   = (32, 32, 64)
    upscales = (True, True, False)
    nunits   = (2, 3, 2)

    x = UpSampling2D(size)(x)
    for i in range(3):
        x = InvResNetBlock(x, nunits[i], depth=depths[i], upscale=upscales[i])
    x = UpSampling2D((2,2))(x)
    x = Conv2DTranspose(3, (7, 7), strides=2, padding='same', use_bias=False)(x)
    return x

In [0]:
def G_init():
    xin = Input(shape=(32,), name="Input")
    x = Reshape((1,1,32))(xin)
    x = InvResNet9(x)
    x = Activation('relu')(x)
    model = Model(xin, x)
    return model

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = 8 #int(np.sqrt(s / r))
            h = 8 #int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = 0.0 #np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
        fill_mode = 'constant',
        cval=0,
        #rotation_range=20,
        #width_shift_range=8,  # randomly shift images horizontally (fraction of total width)
        #height_shift_range=8,  # randomly shift images vertically (fraction of total height)
        #horizontal_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(v_l=min_pix, v_h=max_pix, pixel_level=False)
)
datagen.fit(trainX)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result)
    true_class = np.argmax(test_y)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['bacc'])+1),model_history.history['bacc'])
    axs[0].plot(range(1,len(model_history.history['val_bacc'])+1),model_history.history['val_bacc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['bacc'])+1),len(model_history.history['bacc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [16]:
BATCH_SIZE = 64
WT_DECAY   = 1e-5
MOMENTUM   = 0.90
LEARNING_RATE = 0.002

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras import backend as K

def l2_weights(model):
    l2 = 0
    for layer in model.layers: 
        wt = layer.weights
        if len(wt) > 0:
            l2 += K.sum(K.pow(wt, 2))
    return l2

def reg_loss(model):
    def rloss(y_true, y_pred):
        return WT_DECAY*l2_weights(model)
    return rloss

def loss_with_regularization(model):
    def loss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred, True) + reg_loss(model)(y_true, y_pred)
    return loss

def bacc(y_true, y_pred):
    y_pred = K.cast(y_pred > 0.5, dtype=y_true.dtype)
    return K.mean(K.equal(y_true, y_pred), axis=-1)

Q = Q_init()
optimizer = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)
Q.compile(optimizer=optimizer, loss=loss_with_regularization(Q), metrics=[bacc, reg_loss(Q)])
Q.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 16, 16, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalizatio

In [17]:
D = D_init()

D.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['acc'])
D.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 12289     
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________


In [18]:
G = G_init()

G.compile(optimizer=Adam(0.0002, 0.5), loss='mse', metrics=['acc'])
G.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 1, 32)          0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_14 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 4, 4, 32)          9216      
_________________________________________________________________
batch_normalization_15 (Batc (None, 4, 4, 32)          128 

In [0]:
for i in range(2000):
    Q.fit(trainX, trainY, batch_size=BATCH_SIZE, validation_data = (testX, testY))
    Gin = K.function(Q.input, Q.layers[-1].output)(trainX)
    G.fit(Gin, trainX, batch_size=BATCH_SIZE)
    Gout = G.predict(Gin)
    Dx = np.append(trainX, Gout, axis=0)
    Dy = np.append(np.ones((trainX.shape[0],)), np.zeros((Gout.shape[0],)))
    s = np.arange(Dy.shape[0])
    np.random.shuffle(s)
    Dx, Dy = Dx[s], Dy[s]
    D.fit(Dx, Dy, batch_size=BATCH_SIZE, validation_data = (testX, np.ones((testX.shape[0],))))

Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 7s 1ms/sample - loss: 0.7322 - bacc: 0.4766 - rloss: 0.0179 - val_loss: 0.7201 - val_bacc: 0.4891 - val_rloss: 0.0177
Train on 6750 samples
6750/6750 [==============================] - 3s 382us/sample - loss: 0.9365 - acc: 0.5215
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 85us/sample - loss: 0.3980 - acc: 0.9136 - val_loss: 0.0964 - val_acc: 0.9716
Train on 6750 samples, validate on 670 samples
6750/6750 [==============================] - 2s 284us/sample - loss: 0.6892 - bacc: 0.5117 - rloss: 0.0180 - val_loss: 0.7277 - val_bacc: 0.5125 - val_rloss: 0.0185
Train on 6750 samples
6750/6750 [==============================] - 2s 240us/sample - loss: 0.8780 - acc: 0.5599
Train on 13500 samples, validate on 670 samples
13500/13500 [==============================] - 1s 60us/sample - loss: 0.3523 - acc: 0.9500 - val_loss: 0.2527 - val_acc: 0.9373
Tr